In [ ]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import os
import zipfile
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import VGG16

# Download and extract dataset
url = "https://storage.googleapis.com/mledu-datasets/cats_and_dogs_filtered.zip"
filename = os.path.join(os.getcwd(), "cats_and_dogs_filtered.zip")
tf.keras.utils.get_file(filename, url)

with zipfile.ZipFile("cats_and_dogs_filtered.zip", "r") as zip_ref:
    zip_ref.extractall()

# Define data generators
train_dir = os.path.join(os.getcwd(), "cats_and_dogs_filtered", "train")
validation_dir = os.path.join(os.getcwd(), "cats_and_dogs_filtered", "validation")

train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True
)

validation_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(150, 150),
    batch_size=20,
    class_mode="binary"
)

validation_generator = validation_datagen.flow_from_directory(
    validation_dir,
    target_size=(150, 150),
    batch_size=20,
    class_mode="binary"
)

# Load pre-trained VGG16 model
conv_base = VGG16(
    weights="imagenet",
    include_top=False,
    input_shape=(150, 150, 3)
)

# Freeze convolutional base layers
conv_base.trainable = False

# Build model on top of the convolutional base
model = tf.keras.models.Sequential()
model.add(conv_base)
model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dense(256, activation="relu"))
model.add(tf.keras.layers.Dropout(0.5))
model.add(tf.keras.layers.Dense(1, activation="sigmoid"))

# Compile model
model.compile(
    loss="binary_crossentropy",
    optimizer=tf.keras.optimizers.RMSprop(learning_rate=2e-5),
    metrics=["accuracy"]
)

# Train model
history = model.fit(
    train_generator,
    steps_per_epoch=100,
    epochs=30,
    validation_data=validation_generator,
    validation_steps=50
)

# Show sample input and its predicted class
x, y_true = next(validation_generator)
y_pred = model.predict(x)
class_names = ['cat', 'dog']

for i in range(len(x)) :
    plt.imshow(x[i])
    plt.title(f'Predicted class: {class_names[int(round(y_pred[i][0]))]}, True class: {class_names[int(y_true[i])]}')
    plt.show()

# Plot accuracy and loss over time
acc = history.history["accuracy"]
val_acc = history.history["val_accuracy"]
loss = history.history["loss"]
val_loss = history.history["val_loss"]
epochs = range(1, len(acc) + 1)

plt.plot(epochs, acc, "bo", label="Training acc")
plt.plot(epochs, val_acc, "b", label="Validation acc")
plt.title("Training and validation accuracy")
plt.legend()
plt.figure()
plt.plot(epochs, loss, "bo", label="Training loss")
plt.plot(epochs, val_loss, "b", label="Validation loss")
plt.title("Training and validation loss")
plt.legend()
plt.show()


Found 2000 images belonging to 2 classes.
Found 1000 images belonging to 2 classes.
Epoch 1/30
100/100 [==============================] - 992s 10s/step - loss: 0.6606 - accuracy: 0.6220 - val_loss: 0.4639 - val_accuracy: 0.8350
Epoch 2/30
100/100 [==============================] - 278s 3s/step - loss: 0.5367 - accuracy: 0.7295 - val_loss: 0.3952 - val_accuracy: 0.8350
Epoch 3/30
100/100 [==============================] - 279s 3s/step - loss: 0.4757 - accuracy: 0.7680 - val_loss: 0.3587 - val_accuracy: 0.8510
Epoch 4/30
100/100 [==============================] - 280s 3s/step - loss: 0.4410 - accuracy: 0.7995 - val_loss: 0.3386 - val_accuracy: 0.8570
Epoch 5/30
100/100 [==============================] - 271s 3s/step - loss: 0.4230 - accuracy: 0.7965 - val_loss: 0.3206 - val_accuracy: 0.8610
Epoch 6/30
100/100 [==============================] - 369s 4s/step - loss: 0.4007 - accuracy: 0.8255 - val_loss: 0.3083 - val_accuracy: 0.8660
Epoch 7/30
100/100 [==============================] - 289

In [ ]:
Aim: Implement Transfer Learning on the suitable public dataset (e.g., classify the cats versus dog’s
dataset from Kaggle or UCI or inbuilt dataset).

In [ ]:
Theory: Transfer learning is a machine learning (ML) method that reuses a trained model designed for a
particular task to accomplish a different yet related task. The knowledge acquired from task one is thereby
transferred to the second model that focuses on the new task.
The term ‘transfer learning’ is related to human psychology. For example, consider an individual who is
an expert guitarist. It is quite easy for him to learn to play other stringed instruments, such as a sitar or
mandolin, compared to someone with no experience playing any musical instrument.
Transfer learning speeds up the overall process of training a new model and consequently improves its
performance. It is primarily used when a model requires large amount of resources and time for training.
Due to these reasons, transfer learning is employed in several deep learning projects, such as neural
networks that accomplish NLP or CV tasks, such as sentiment analysis.